In [ ]:
# clone repo and install dependences
!pip install --upgrade --no-cache-dir gdown
from IPython.display import clear_output
!git clone https://github.com/siidev/StyleMapGAN.git
%cd StyleMapGAN/
!pip install ninja
!pip install flask-ngrok
!pip install scikit-image==0.16.2
clear_output()
print('Done!')

Done!


In [ ]:
# download lmdb data
!gdown --id 1J1bWS1C51NV7VrQX8R9xR1XNd6P5jD1c 
!unzip vn_celeb.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1J1bWS1C51NV7VrQX8R9xR1XNd6P5jD1c
To: /content/StyleMapGAN/vn_celeb.zip
100% 268M/268M [00:02<00:00, 112MB/s]
Archive:  vn_celeb.zip
   creating: lmdb-data/
   creating: lmdb-data/vn_celeb/
   creating: lmdb-data/vn_celeb/test/
  inflating: lmdb-data/vn_celeb/test/data.mdb  
  inflating: lmdb-data/vn_celeb/test/lock.mdb  
   creating: lmdb-data/vn_celeb/train/
  inflating: lmdb-data/vn_celeb/train/data.mdb  
  inflating: lmdb-data/vn_celeb/train/lock.mdb  
   creating: lmdb-data/vn_celeb/val/
  inflating: lmdb-data/vn_celeb/val/data.mdb  
  inflating: lmdb-data/vn_celeb/val/lock.mdb  


In [ ]:
# prepare fid
import gdown
import os

url = 'https://drive.google.com/uc?id=1pCr4lNCON7IZcNVdskIDXhFJ3jYuge1w' # inception checkpoint
output = 'metrics/pt_inception-2015-12-05-6726825d.pth'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1dMHaNkgIy_-cq70f6nqoQsfG9-aOTpSC' # random generated samples as pickle files
output = 'fid_stats.zip'
gdown.download(url, output, quiet=False)
!unzip fid_stats.zip

Downloading...
From: https://drive.google.com/uc?id=1pCr4lNCON7IZcNVdskIDXhFJ3jYuge1w
To: /content/StyleMapGAN/metrics/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 95.6M/95.6M [00:00<00:00, 209MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dMHaNkgIy_-cq70f6nqoQsfG9-aOTpSC
To: /content/StyleMapGAN/fid_stats.zip
100%|██████████| 96.3M/96.3M [00:01<00:00, 68.1MB/s]


Archive:  fid_stats.zip
   creating: metrics/fid_stats/
  inflating: metrics/fid_stats/celeba_hq_stats_256_29000.pkl  
  inflating: metrics/fid_stats/afhq_stats_256_15130.pkl  
  inflating: metrics/fid_stats/vn_celeb_stats_128_22105.pkl  


In [ ]:
# download checkpoint
checkpoint = '200000.pt'
!gdown --id 1SHCwwyOnJUpk4N8fCNARXHKqG3HqSEDK

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1SHCwwyOnJUpk4N8fCNARXHKqG3HqSEDK
To: /content/StyleMapGAN/200000.pt
100% 1.02G/1.02G [00:09<00:00, 111MB/s]


In [ ]:
# Reconstruction
!python generate.py --ckpt {checkpoint} --mixing_type reconstruction --test_lmdb lmdb-data/vn_celeb/test
!python -m metrics.reconstruction --image_folder_path expr/reconstruction/vn_celeb

Namespace(batch=16, batch_per_gpu=8, channel_multiplier=2, ckpt='378000.pt', d_reg_every=16, dataset='vn_celeb', iter=1400000, lambda_adv_loss=1, lambda_d_loss=1, lambda_indomainGAN_D_loss=1, lambda_indomainGAN_E_loss=1, lambda_perceptual_loss=1, lambda_w_rec_loss=1, lambda_x_rec_loss=1, latent_channel_size=64, latent_spatial_size=8, local_editing_part=None, lr=0.002, lr_mul=0.01, mapping_layer_num=8, mixing_type='reconstruction', ngpus=1, normalize_mode='LayerNorm', num_workers=10, r1=10, save_image_dir='expr', save_network_interval=2000, size=128, small_generator=True, start_iter=368000, test_lmdb='lmdb-data/vn_celeb/test', train_data='lmdb-data/vn_celeb/train', val_data='lmdb-data/vn_celeb/val')
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that exc

In [ ]:
# !python -m metrics.fid --generated_image_path lmdb-data/vn_celeb/train --dataset vn_celeb --size 128
!python -m metrics.fid --ckpt {checkpoint} --comparative_fid_pkl metrics/fid_stats/vn_celeb_stats_128_22105.pkl --size 128 --dataset vn_celeb

/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:83: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
  0% 0/1382 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100% 1382/1382 [34:26<00:00,  1.50s/it]
extracted 22105 features
fid: 8.013711098501972


In [ ]:
# Reconstruction
!python generate.py --ckpt {checkpoint} --mixing_type reconstruction --test_lmdb lmdb-data/vn_celeb/test
!python -m metrics.reconstruction --image_folder_path expr/reconstruction/vn_celeb
!python -m metrics.fid --ckpt {checkpoint} --comparative_fid_pkl metrics/fid_stats/vn_celeb_stats_128_22105.pkl --size 128 --dataset vn_celeb

Namespace(batch=16, batch_per_gpu=8, channel_multiplier=2, ckpt='200000.pt', d_reg_every=16, dataset='vn_celeb', iter=1400000, lambda_adv_loss=1, lambda_d_loss=1, lambda_indomainGAN_D_loss=1, lambda_indomainGAN_E_loss=1, lambda_perceptual_loss=1, lambda_w_rec_loss=1, lambda_x_rec_loss=1, latent_channel_size=64, latent_spatial_size=8, local_editing_part=None, lr=0.002, lr_mul=0.01, mapping_layer_num=8, mixing_type='reconstruction', ngpus=1, normalize_mode='LayerNorm', num_workers=10, r1=10, save_image_dir='expr', save_network_interval=2000, size=128, small_generator=True, start_iter=192000, test_lmdb='lmdb-data/vn_celeb/test', train_data='lmdb-data/vn_celeb/train', val_data='lmdb-data/vn_celeb/val')
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that exc

In [ ]:
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/vn_celeb/test
!python -m metrics.local_editing --data_dir expr/local_editing/vn_celeb

Namespace(batch=16, batch_per_gpu=8, channel_multiplier=2, ckpt='200000.pt', d_reg_every=16, dataset='vn_celeb', iter=1400000, lambda_adv_loss=1, lambda_d_loss=1, lambda_indomainGAN_D_loss=1, lambda_indomainGAN_E_loss=1, lambda_perceptual_loss=1, lambda_w_rec_loss=1, lambda_x_rec_loss=1, latent_channel_size=64, latent_spatial_size=8, local_editing_part=None, lr=0.002, lr_mul=0.01, mapping_layer_num=8, mixing_type='local_editing', ngpus=1, normalize_mode='LayerNorm', num_workers=10, r1=10, save_image_dir='expr', save_network_interval=2000, size=128, small_generator=True, start_iter=192000, test_lmdb='lmdb-data/vn_celeb/test', train_data='lmdb-data/vn_celeb/train', val_data='lmdb-data/vn_celeb/val')
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that exce

parts ['']

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100% 63/63 [00:04<00:00, 15.57it/s]
0.030, 0.029
average in inMSE_ref, outMSE_src
0.030, 0.029
